In [1]:
import json
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
pd.options.display.max_rows = 20
pd.options.display.max_columns = 999

In [3]:
#get review details
def get_review_level_df(product_id,file_name):
    review_level_list = ['Id','ProductId','Rating','Title','ReviewText', 'IsRecommended','SubmissionTime',
                        'ContextDataValues.skinType.ValueLabel','ContextDataValues.eyeColor.ValueLabel','ContextDataValues.hairColor.ValueLabel',
                        'ContextDataValues.skinTone.ValueLabel','ContextDataValues.age.ValueLabel', 'ContextDataDistribution.skinConcerns.ValueLabel']
    df = pd.json_normalize(file_name[product_id][1])
    for i in review_level_list:
        if i not in df:
            df[i] = np.nan
    df_review_level = df.loc[:,review_level_list]
    df_review_level['Product_id_original'] = product_id
    return df_review_level

In [4]:
#get review_product 
def get_review_product_df(product_id,file_name):
    review_product_list = ['AverageOverallRating','FirstSubmissionTime', 'RecommendedCount', 'NotRecommendedCount','RatingDistribution',
                           'ContextDataDistribution.age.Values','ContextDataDistribution.skinType.Values','ContextDataDistribution.skinConcerns.Values',
                           'ContextDataDistribution.skinTone.Values','ContextDataDistribution.eyeColor.Values', 
                           'ContextDataDistribution.hairColor.Values']
    
    df = pd.DataFrame(file_name[product_id][0])  
    df= df.transpose().reset_index()
    df['Product_id']=df['Id']
    df['Product_id_original'] = product_id
    df_header_list=['Product_id_original', 'Product_id','Name','Brand','BrandExternalId','CategoryId','TotalReviewCount','ReviewStatistics']
    df_header = df.loc[:,df_header_list].copy()
    
    df = pd.json_normalize(df['ReviewStatistics'])
    for i in review_product_list:
        if i not in df:
            df[i] = np.nan
    df_review_product = df.loc[:,review_product_list]
    
    df_review_product = pd.concat([df_header, df_review_product], axis = 1)
    
    return df_review_product

In [5]:
#extract all product
def extract_all_product(file_name):
    products = list(file_name.keys())
    
    review_product_lst = []
    review_level_lst = []
    
    for product_id in products:
#         print(product_id)
        if file_name[product_id] != [[], []] and file_name[product_id] != [None, None]:
            tmp1 = get_review_product_df(product_id,file_name)
            review_product_lst.append(tmp1)
            tmp2 = get_review_level_df(product_id,file_name)
            review_level_lst.append(tmp2)
    
    df_review_product = pd.concat(review_product_lst)
    df_review_level = pd.concat(review_level_lst)
    return [df_review_product, df_review_level]

In [6]:
product_lst = []
review_lst = []
files = ['scraper_result'+str(i+1) for i in range(0, 13)]

for file in tqdm(files):
    file12 = None
    with open(f'data/{file}.json', 'rb') as file:
        file12 = json.load(file)
    
    df_review_product, df_review_level = extract_all_product(file12)
    product_lst.append(df_review_product)
    review_lst.append(df_review_level)
    del file12

100%|██████████| 13/13 [32:57<00:00, 152.15s/it]


In [7]:
df_product = pd.concat(product_lst)
df_review = pd.concat(review_lst)

In [ ]:
del product_lst
del review_lst

In [8]:
len(df_product)

14789

In [9]:
len(df_review)

2700551

In [10]:
df_product.to_pickle('data/all_products.pkl')

In [11]:
df_review.to_pickle('data/all_reviews.pkl')

# Appendix

In [76]:
#Example 
get_review_product_df('P417867',file12)

,Product_id,Name,Brand,BrandExternalId,CategoryId,TotalReviewCount,ReviewStatistics,AverageOverallRating,RecommendedCount,NotRecommendedCount,RatingDistribution,ContextDataDistribution.age.Values,ContextDataDistribution.skinType.Values,ContextDataDistribution.skinConcerns.Values,ContextDataDistribution.skinTone.Values,ContextDataDistribution.eyeColor.Values,ContextDataDistribution.hairColor.Values
0,P417867,The Silk Cream,"{'Id': '6041', 'Name': 'Tatcha'}",6041,cat60097,241,"{'FeaturedReviewCount': 0, 'RecommendedCount':...",4.2158,172,49,"[{'RatingValue': 1, 'Count': 21}, {'RatingValu...","[{'Value': '13to17', 'Count': 1}, {'Value': '1...","[{'Value': 'normal', 'Count': 35}, {'Value': '...","[{'Value': 'acne', 'Count': 79}, {'Value': 'ag...","[{'Value': 'fair', 'Count': 52}, {'Value': 'li...","[{'Value': 'blue', 'Count': 30}, {'Value': 'br...","[{'Value': 'blonde', 'Count': 39}, {'Value': '..."


In [4]:
with open('data/scraper_result2.json', 'r') as file:
    file2 = json.load(file)

In [4]:
file2.keys()

dict_keys(['P270551', 'P378145', 'P270548', 'P270549', 'P431505', 'P454116', 'P385586', 'P382581', 'P270600', 'P404241', 'P270544', 'P388664', 'P270540', 'P390422', 'P270543', 'P439913', 'P270545', 'P270570', 'P270559', 'P400625', 'P422224', 'P385587', 'P383334', 'P270541', 'P395372', 'P270542', 'P422076', 'P406621', 'P309704', 'P434383', 'P403811', 'P432843', 'P270562', 'P270574', 'P432842', 'P294810', 'P390431', 'P401774', 'P432629', 'P270568', 'P270560', 'P270591', 'P199612', 'P266800', 'P254604', 'P302940', 'P61204', 'P393292', 'P432254', 'P387006', 'P404425', 'P434359', 'P387135', 'P425899', 'P417305', 'P388628', 'P457415', 'P418507', 'P446343', 'P396733', 'P440273', 'P429962', 'P418506', 'P431544', 'P402071', 'P411359', 'P388627', 'P402073', 'P402074', 'P388626', 'P388625', 'P440274', 'P449134', 'P429961', 'P427526', 'P446341', 'P446340', 'P402072', 'P418505', 'P425409', 'P449133', 'P396734', 'P435402', 'P444595', 'P449577', 'P443755', 'P443756', 'P386464', 'P280566', 'P386462', 

In [12]:
pd.json_normalize(file2['P410669'][1])

,UserNickname,Rating,ReviewText,ProductId,UserEmailAddress,Videos,CommentIds,Photos,TotalNegativeFeedbackCount,RatingRange,CampaignId,UserLocation,SecondaryRatingsOrder,TotalCommentCount,ContextDataValuesOrder,ClientResponses,TagDimensionsOrder,TotalPositiveFeedbackCount,IsRatingsOnly,LastModificationTime,Pros,ProductRecommendationIds,BadgesOrder,AuthorId,Helpfulness,SubmissionTime,AdditionalFieldsOrder,TotalFeedbackCount,Title,ModerationStatus,IsFeatured,Cons,SubmissionId,IsSyndicated,IsRecommended,LastModeratedTime,Id,ContentLocale,ContextDataValues.skinType.Value,ContextDataValues.skinType.Id,ContextDataValues.skinType.ValueLabel,ContextDataValues.skinType.DimensionLabel,ContextDataValues.eyeColor.Value,ContextDataValues.eyeColor.Id,ContextDataValues.eyeColor.ValueLabel,ContextDataValues.eyeColor.DimensionLabel,ContextDataValues.StaffContext.Value,ContextDataValues.StaffContext.Id,ContextDataValues.StaffContext.ValueLabel,ContextDataValues.StaffContext.DimensionLabel,ContextDataValues.VerifiedPurchaser.Value,ContextDataValues.VerifiedPurchaser.Id,ContextDataValues.VerifiedPurchaser.ValueLabel,ContextDataValues.VerifiedPurchaser.DimensionLabel,ContextDataValues.hairColor.Value,ContextDataValues.hairColor.Id,ContextDataValues.hairColor.ValueLabel,ContextDataValues.hairColor.DimensionLabel,ContextDataValues.skinTone.Value,ContextDataValues.skinTone.Id,ContextDataValues.skinTone.ValueLabel,ContextDataValues.skinTone.DimensionLabel,ContextDataValues.IncentivizedReview.Value,ContextDataValues.IncentivizedReview.Id,ContextDataValues.IncentivizedReview.ValueLabel,ContextDataValues.IncentivizedReview.DimensionLabel,AdditionalFields.sociallockup.Label,AdditionalFields.sociallockup.Value,AdditionalFields.sociallockup.Id,Badges.VerifiedPurchaser.ContentType,Badges.VerifiedPurchaser.Id,Badges.VerifiedPurchaser.BadgeType,ContextDataValues.age.Value,ContextDataValues.age.Id,ContextDataValues.age.ValueLabel,ContextDataValues.age.DimensionLabel,ContextDataValues.beautyInsider.Value,ContextDataValues.beautyInsider.Id,ContextDataValues.beautyInsider.ValueLabel,ContextDataValues.beautyInsider.DimensionLabel,TagDimensions.Pro.Label,TagDimensions.Pro.Values,TagDimensions.Pro.Id,Badges.yesvibr.ContentType,Badges.yesvibr.Id,Badges.yesvibr.BadgeType,Badges.yesvib.ContentType,Badges.yesvib.Id,Badges.yesvib.BadgeType,Badges.beautyInsider.ContentType,Badges.beautyInsider.Id,Badges.beautyInsider.BadgeType,Badges.IncentivizedReviewBadge.ContentType,Badges.IncentivizedReviewBadge.Id,Badges.IncentivizedReviewBadge.BadgeType,Badges.StaffContextBadge.ContentType,Badges.StaffContextBadge.Id,Badges.StaffContextBadge.BadgeType
0,sharkita,5,I thought I would finally try something Armani...,1857879,None,[],[],[],0,5,None,None,[],0,"[eyeColor, hairColor, skinTone, skinType, Ince...",[],[],1,False,2021-05-08T02:12:29.000+00:00,None,[],[VerifiedPurchaser],1078075075,1.000000,2021-05-06T03:34:32.000+00:00,[sociallockup],1,None,APPROVED,False,None,r18723api-_16202720jjo7xNZwPj,False,True,2021-05-06T04:00:22.000+00:00,179190081,en_US,combination,skinType,Combination,Skin Type,hazel,eyeColor,Hazel,Eye Color,false,StaffContext,No,I am a Sephora employee,True,VerifiedPurchaser,Yes,Verified Purchaser,brunette,hairColor,Brunette,Hair color,light,skinTone,Light,Skin Tone,false,IncentivizedReview,False,I received this product as a free sample,sociallockup,avatar=https://community.sephora.com/t5/image/...,sociallockup,REVIEW,VerifiedPurchaser,Custom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Rani2017,5,This Mascara is everything it makes my lashes ...,1857879,None,[],[],[],0,5,None,None,[],0,"[eyeColor, hairColor, skinTone, skinType, Ince...",[],[],0,False,2021-04-26T05:00:16.000+00:00,None,[],[],5636350033,NaN,2021-04-26T04:41:43.000+00:00,[sociallockup],0,None,APPROVED,False,None,r18723api-_16194121Uzahr5PPkc,False,True,2021-04-26T05:00:16.000+00:00,178543958,en_CA,dry,skinType,Dry,Skin Type,brown,eyeColor,Brown

In [6]:
df = pd.DataFrame(file2['P410669'][1])

In [7]:
df.columns

Index(['UserNickname', 'Rating', 'ReviewText', 'ContextDataValues',
       'ProductId', 'UserEmailAddress', 'Videos', 'CommentIds', 'Photos',
       'TotalNegativeFeedbackCount', 'RatingRange', 'CampaignId',
       'UserLocation', 'SecondaryRatingsOrder', 'TotalCommentCount',
       'ContextDataValuesOrder', 'ClientResponses', 'TagDimensionsOrder',
       'TotalPositiveFeedbackCount', 'IsRatingsOnly', 'LastModificationTime',
       'Pros', 'ProductRecommendationIds', 'BadgesOrder', 'AuthorId',
       'Helpfulness', 'SubmissionTime', 'AdditionalFields',
       'AdditionalFieldsOrder', 'TotalFeedbackCount', 'Title',
       'ModerationStatus', 'IsFeatured', 'Cons', 'TagDimensions',
       'SecondaryRatings', 'Badges', 'SubmissionId', 'IsSyndicated',
       'IsRecommended', 'LastModeratedTime', 'Id', 'ContentLocale'],
      dtype='object')

In [8]:
df = pd.json_normalize(file2['P410669'][1])

In [11]:
df[['Id','ProductId','Rating','Title','ReviewText', 'IsRecommended','SubmissionTime',
                        'ContextDataValues.skinType.ValueLabel','ContextDataValues.eyeColor.ValueLabel','ContextDataValues.hairColor.ValueLabel',
                        'ContextDataValues.skinTone.ValueLabel','ContextDataValues.age.ValueLabel']]

,Id,ProductId,Rating,Title,ReviewText,IsRecommended,SubmissionTime,ContextDataValues.skinType.ValueLabel,ContextDataValues.eyeColor.ValueLabel,ContextDataValues.hairColor.ValueLabel,ContextDataValues.skinTone.ValueLabel,ContextDataValues.age.ValueLabel
0,179190081,1857879,5,None,I thought I would finally try something Armani...,True,2021-05-06T03:34:32.000+00:00,Combination,Hazel,Brunette,Light,NaN
1,178543958,1857879,5,None,This Mascara is everything it makes my lashes ...,True,2021-04-26T04:41:43.000+00:00,Dry,Brown,Brunette,Medium,NaN
2,177535670,1857879,1,DO NOT BUY,This mascara is HORIBLE. It comes out of your ...,False,2021-04-10T11:56:58.000+00:00,Combination,Green,Blonde,Porcelain,NaN
3,169010966,1857879,3,None,This didn’t do much for me. Clumped my lashes ...,False,2020-12-15T20:05:52.000+00:00,Normal,Brown,Brunette,Olive,NaN
4,166370515,1857879,5,Best mascara,Most amazing mascara. I have most ordinary las...,True,2020-11-28T01:07:14.000+00:00,Normal,Brown,Brunette,Medium,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
170,72392533,P410669,5,LOVEEE !!,Beyond impressed with this mascara !! I receiv...,None,2016-08-26T02:58:39.000+00:00,NaN,NaN,NaN,NaN,NaN
171,72342123,P410669,5,Impressed!,"I received this product as a sample, which was...",None,2016-08-25T18:49:46.000+00:00,NaN,Blue,NaN,Light,25-34
172,72339540,P410669,5,one of the best mascaras ever,Okay you need this mascara there's no doubt ab...,None,2016-08-25T18:05:10.000+00:00,NaN,NaN,NaN,NaN,NaN
173,71491005,P410669,5,Perfect volume!,This mascara gives the perfect amount of volum...,None,2016-08-12T01:14:13.000+00:00,NaN,Brown,NaN,Deep,25-34


In [10]:
file2['P385175'][1]

[{'UserNickname': 'Lizz1126',
  'Rating': 5,
  'ReviewText': 'My husband tried this cologne, it was a subtle but pleasant smell. Easy on the nose. I do however recommend this for older gentlemen since my husband himself said he preferred something with a stronger smell. Overall he says he would wear it and enjoys the smell.',
  'ContextDataValues': {'skinType': {'Value': 'normal',
    'Id': 'skinType',
    'ValueLabel': 'Normal',
    'DimensionLabel': 'Skin Type'},
   'eyeColor': {'Value': 'brown',
    'Id': 'eyeColor',
    'ValueLabel': 'Brown',
    'DimensionLabel': 'Eye Color'},
   'StaffContext': {'Value': 'false',
    'Id': 'StaffContext',
    'ValueLabel': 'No',
    'DimensionLabel': 'I am a Sephora employee'},
   'hairColor': {'Value': 'brunette',
    'Id': 'hairColor',
    'ValueLabel': 'Brunette',
    'DimensionLabel': 'Hair color'},
   'skinTone': {'Value': 'medium',
    'Id': 'skinTone',
    'ValueLabel': 'Medium',
    'DimensionLabel': 'Skin Tone'},
   'IncentivizedReview':

In [14]:
pd(file2['P385175'][0])

{'P385175': {'CategoryId': 'cat60148',
  'EANs': ['3700591213007', '3700591213014', '3700591213021', '3700591213038'],
  'Description': 'The launch of Cédrat Enivrant celebrates a return to the origins of Atelier Cologne, the first maison de parfum dedicated entirely to cologne, the legendary fragrance classic. The creators, Sylvie Ganter and Christophe Cervasel gave birth to a new olfactive family, “Cologne Absolue”. Cédrat Enivrant completes their Collection Originale, now composed of six new colognes.  Designed primarily around citrus notes—the traditional ingredients of a cologne—and inspired by the French 75 cocktail, Cédrat Enivrant is for lovers of lemon and gin notes.\n\nNotes:\nMaroccan Cedrat, Lime, Bergamot, Chinese Mint, Egyptian Basilic,\nJuniper Berries, Tonka Bean, Vetiver, Elemi.\n\nStyle:\nSophisticated. Effortless. Classic.',
  'ModelNumbers': [],
  'Attributes': {'BV_FE_EXPAND': {'Values': [{'Locale': None,
      'Value': 'BV_FE_FAMILY:P385175'}],
    'Id': 'BV_FE_EX